In [105]:
# Data import using Spark ML
from pyspark.sql import SQLContext
#from pyspark.sql.types import *
import pandas as pd
from pyspark.ml import Pipeline
#from pyspark.ml.feature import StringIndexer, VectorIndexer

In [106]:
f = pd.read_csv("daily_weather.csv", sep=',', index_col=0)

In [131]:
f.head()

,air_pressure_9am,air_temp_9am,avg_wind_direction_9am,avg_wind_speed_9am,max_wind_direction_9am,max_wind_speed_9am,rain_accumulation_9am,rain_duration_9am,label
0,921.551868,67.619839,197.965342,4.026048,210.834658,4.902075,1.020587e-14,0.0,1
1,923.100000,75.596000,178.300000,1.409272,205.200000,1.856660,3.069545e-13,0.0,1
2,914.970000,73.688000,188.200000,2.259309,186.600000,3.064608,0.000000e+00,0.0,0
3,916.610000,72.338000,192.100000,1.834291,232.400000,2.572481,0.000000e+00,0.0,0
4,919.850000,77.360000,235.200000,1.879030,222.800000,2.505373,0.000000e+00,0.0,0


In [108]:
del f['relative_humidity_9am']

In [109]:
def functionG(row):
    if row['relative_humidity_3pm'] < 25:
        val = 1
    else:
        val = 0
    return val

In [110]:
# Convert to Format (label, features)
#g = f.copy(deep=True)
#low_humidity = pd.DataFrame()
f['label'] = f.apply(functionG, axis=1)

In [111]:
del f['relative_humidity_3pm']

In [112]:
f.columns[0:]
#temp = f.map(lambda line:LabeledPoint(line[0],[line[1:]]))

Index([u'air_pressure_9am', u'air_temp_9am', u'avg_wind_direction_9am',
       u'avg_wind_speed_9am', u'max_wind_direction_9am', u'max_wind_speed_9am',
       u'rain_accumulation_9am', u'rain_duration_9am', u'label'],
      dtype='object')

In [113]:
#from pyspark.mllib.linalg import DenseVector 
cols = ['air_pressure_9am','air_temp_9am','avg_wind_direction_9am','avg_wind_speed_9am','max_wind_direction_9am','max_wind_speed_9am','rain_accumulation_9am','rain_duration_9am']

In [114]:
#from pyspark.mllib.linalg import Vectors, VectorUDT

#w = pd.DataFrame()
#w['fea'] = f[cols].apply(lambda x:','.join(x.map(str)), axis=1)

#w['fea'] = [ f[c] for c in cols ]
#w.head(10)

In [115]:
from pyspark.ml.classification import DecisionTreeClassifier

In [117]:
#data = pd.concat([low_humidity, f], axis = 1)
#data.head(1)
data = f.copy(deep=True)
data.head(2)

,air_pressure_9am,air_temp_9am,avg_wind_direction_9am,avg_wind_speed_9am,max_wind_direction_9am,max_wind_speed_9am,rain_accumulation_9am,rain_duration_9am,label
0,921.551868,67.619839,197.965342,4.026048,210.834658,4.902075,1.020587e-14,0.0,1
1,923.100000,75.596000,178.300000,1.409272,205.200000,1.856660,3.069545e-13,0.0,1


In [118]:
sqlContext = SQLContext(sc)
dataDF = sqlContext.createDataFrame(data.dropna())
dataDF.take(2)

[Row(air_pressure_9am=921.5518680014079, air_temp_9am=67.61983914669104, avg_wind_direction_9am=197.9653423773277, avg_wind_speed_9am=4.026047646630689, max_wind_direction_9am=210.83465762267227, max_wind_speed_9am=4.9020753052407615, rain_accumulation_9am=1.0205866197819671e-14, rain_duration_9am=0.0, label=1),
 Row(air_pressure_9am=923.1000000000024, air_temp_9am=75.5960000000033, avg_wind_direction_9am=178.3, avg_wind_speed_9am=1.4092721999998965, max_wind_direction_9am=205.19999999999996, max_wind_speed_9am=1.856660199999978, rain_accumulation_9am=3.069544618483633e-13, rain_duration_9am=0.0, label=1)]

# Features Column

In [119]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=cols, outputCol="features")
assembled = assembler.transform(dataDF)
assembled

DataFrame[air_pressure_9am: double, air_temp_9am: double, avg_wind_direction_9am: double, avg_wind_speed_9am: double, max_wind_direction_9am: double, max_wind_speed_9am: double, rain_accumulation_9am: double, rain_duration_9am: double, label: bigint, features: vector]

In [120]:
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(assembled)

# Automatically identify categorical features, and index them.
# We specify maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer= VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(assembled)

# Test and Training Data

In [122]:
# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = assembled.randomSplit([0.7, 0.3])

# Decision Tree in Spark

In [123]:
# Train a DecisionTree model.
dt = DecisionTreeClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures")

# Chain indexers and tree in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, dt])

# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)


In [124]:
# Select example rows to display.
predictions.select("prediction", "indexedLabel", "features").show(5)

+----------+------------+--------------------+
|prediction|indexedLabel|            features|
+----------+------------+--------------------+
|       1.0|         1.0|[910.210000000004...|
|       1.0|         0.0|[911.000000000003...|
|       1.0|         1.0|[911.270000000007...|
|       1.0|         1.0|[911.510000000004...|
|       1.0|         0.0|[912.255790997221...|
+----------+------------+--------------------+
only showing top 5 rows



In [126]:
# Accuracy
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="precision")

accuracy = evaluator.evaluate(predictions)
print("Accuracy = %g " % (accuracy))

Accuracy = 0.801858 


In [128]:
treeModel = model.stages[2]
# summary only
print(treeModel)

DecisionTreeClassificationModel (uid=DecisionTreeClassifier_465eb8e9293200735f89) of depth 5 with 57 nodes


# Naive Bayes in Spark

In [129]:
#Naive Bayes
from pyspark.ml.classification import NaiveBayes 
dtnb = NaiveBayes(labelCol="indexedLabel", featuresCol="indexedFeatures")

# Chain indexers and tree in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, dtnb])

# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)
predictions.select("prediction", "indexedLabel", "features").show(5)

from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="precision")

accuracy = evaluator.evaluate(predictions)
print("Accuracy = %g " % (accuracy))

+----------+------------+--------------------+
|prediction|indexedLabel|            features|
+----------+------------+--------------------+
|       1.0|         1.0|[910.210000000004...|
|       0.0|         0.0|[911.000000000003...|
|       0.0|         1.0|[911.270000000007...|
|       0.0|         1.0|[911.510000000004...|
|       0.0|         0.0|[912.255790997221...|
+----------+------------+--------------------+
only showing top 5 rows

Accuracy = 0.486068 


In [ ]:
# Data cleaning (removing missing values) - check summary statistics

In [ ]:
# Data Cleaning (replace with mean value) - check summary statistics

In [ ]:
# Summary Statistics


In [130]:
# Plots:

# Histogram
# Scatter Plot
# Bar Plot
# Box Plot


In [ ]:
# Data preparation (adding new columns)


In [ ]:
# Declaring Features and Target Class


In [ ]:
# Random Data splitting into test and train


In [ ]:
# Training the Decision Tree Classifier


In [ ]:
# Predicting 


In [ ]:
# Check Accuracy and Confusion Matrix of Decision Tree

In [ ]:
# Training the Naive Bayes Classifier


In [ ]:
# Predicting

In [ ]:
# Check Accuracy and Confusion Matrix of Decision Tree